<a href="https://colab.research.google.com/github/Gonzalo-Messina/Programacion-Concurrente/blob/main/M6_TP1_P4_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio C++

Crear un programa que dado un conjunto de valores almacenados en un array realice la suma de los mismos. Para esto crear dos hilos que se dividirán el array y realizarán la suma del rango que le corresponda almacenando los resultados en una variable compartida, la cual será informada por el
proceso principal al finalizar la ejecución.


In [ ]:
%%writefile m6_mutex.cpp
#include <iostream>
#include <thread>
#include <mutex>

using namespace std;

const int tamano_array = 10;
int array_datos[tamano_array];
int resultado = 0;
mutex mtx;

void sumar_datos(int inicio, int fin) 
{
    int suma_local = 0;
    for (int i = inicio; i < fin; i++) 
    {
      suma_local += array_datos[i];
    }
    mtx.lock();
    resultado += suma_local;
    mtx.unlock();
}

int main() 
{
    // Inicializar array con numeros random entre 0 y 100
    cout << "El array esta compuesto por los siguientes numeros:" << endl;
    for (int i = 0; i < tamano_array; i++) 
    {
      int random = rand() % 101;
      array_datos[i] = random;
      cout << random << " ";
    }

    // Crear 2 hilos dividiento el array
    thread hilo1(sumar_datos, 0, tamano_array/2);
    thread hilo2(sumar_datos, tamano_array/2, tamano_array);

    hilo1.join();
    hilo2.join();

    cout << endl << "La suma es: " << resultado << endl;

    return 0;
}


Overwriting m6_mutex.cpp


In [ ]:
!g++ -std=c++11 -pthread m6_mutex.cpp -o m6_mutex

In [ ]:
!./m6_mutex

El array esta compuesto por los siguientes numeros:
32 32 54 12 52 56 8 30 44 94 
La suma es: 414


# Ejercicio en Java

Crear un programa que genere dos hilos, un productor y un consumidor, el productor producirá un valor entero (0 a 99) y se lo comunicará al consumidor a traves de una variable compartida entera. 

La idea es que exista una alternancia estricta entre el productor y el consumidor, además ambos hilos deberán comunicar por pantalla cada vez que producen/consumen y el valor correspondiente.
Programar dicho comportamiento para producir 10 valores. 

In [ ]:
%%writefile ProducerConsumer.java
public class ProducerConsumer 
{
    private static final Object lock = new Object();
    private static int sharedValue = -1;
    private static boolean isProduced = false;

    public static void main(String[] args) 
    {
        Thread producerThread = new Thread(() -> 
        {
            for (int i = 0; i < 10; i++) 
            {
                synchronized (lock) 
                {
                    while (isProduced) 
                    {
                        try 
                        {
                            lock.wait();
                        } 
                        catch (InterruptedException e) 
                        {
                            e.printStackTrace();
                        }
                    }
                    sharedValue = (int) (Math.random() * 100);
                    System.out.println("Produced: " + sharedValue);
                    isProduced = true;
                    lock.notify();
                }
            }
        });

        Thread consumerThread = new Thread(() -> 
        {
            for (int i = 0; i < 10; i++) 
            {
                synchronized (lock) 
                {
                    while (!isProduced) 
                    {
                        try 
                        {
                            lock.wait();
                        } 
                        catch (InterruptedException e) 
                        {
                            e.printStackTrace();
                        }
                    }
                    System.out.println("Consumed: " + sharedValue);
                    isProduced = false;
                    lock.notify();
                }
            }
        });

        producerThread.start();
        consumerThread.start();
    }
}

Overwriting ProducerConsumer.java


In [ ]:
!javac ProducerConsumer.java

In [ ]:
!java ProducerConsumer

Produced: 70
Consumed: 70
Produced: 35
Consumed: 35
Produced: 90
Consumed: 90
Produced: 51
Consumed: 51
Produced: 71
Consumed: 71
Produced: 97
Consumed: 97
Produced: 71
Consumed: 71
Produced: 66
Consumed: 66
Produced: 48
Consumed: 48
Produced: 8
Consumed: 8


# Ejercicio Python

In [ ]:
%%writefile generar_password.py
import threading
import sys

NUM_HILOS = 2

def convertir_caracteres(cadena, inicio, fin, semaforo, password):
    for i in range(inicio, fin):
        caracter = cadena[i]
        if caracter.islower():
            caracter = caracter.upper()
        num = ord(caracter) - 64
        
        #adquiero y libero el semaforo para acceder al recurso compartido: password
        semaforo.acquire()
        password[i] = num
        semaforo.release()

if __name__ == '__main__':
    
    #valido que se ingrese una cadena a codificar
    if len(sys.argv) != 2:
        print("Debe ingresar una cadena de caracteres como parámetro")
        sys.exit(1)

    cadena = sys.argv[1]

    n = len(cadena)
    password = [0] * n
    hilos = []
    semaforo = threading.Semaphore(1)
    elementos_por_hilo = n // NUM_HILOS

    for i in range(NUM_HILOS):
        inicio = i * elementos_por_hilo
        fin = inicio + elementos_por_hilo
        if i == NUM_HILOS - 1:
            fin = n
        hilo = threading.Thread(target=convertir_caracteres, args=(cadena, inicio, fin, semaforo, password))
        hilo.start()
        hilos.append(hilo)

    for hilo in hilos:
        hilo.join()

    print("La contraseña generada es:", password)

Writing generar_password.py


In [ ]:
!python generar_password.py ARBOLITO

La contraseña generada es: [1, 18, 2, 15, 12, 9, 20, 15]
